In [ ]:
!pip install tmdbsimple

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os, json, math, time
import tmdbsimple as tmdb
from tqdm.notebook import tqdm_notebook 
import gzip

In [2]:
with open('/Users/laura/.tmdbkey/tmdb_api.json','r') as f: 
    login = json.load(f)
login.keys()

dict_keys(['Client-id', 'API-Key'])

In [3]:
tmdb.API_KEY = login['API-Key']

In [ ]:
movie = tmdb.Movies(603)

In [ ]:
movie

In [ ]:
info = movie.info()

In [ ]:
movie = tmdb.Movies('tt1361336')
info = movie.info()
info['budget']


In [ ]:
releases = movie.releases()
for c in releases['countries']:
    if c['iso_3166_1'] == 'US':
        print(c['certification'])

In [ ]:
movie = tmdb.Movies('tt1361336')
# save the .info .releases dictionaries
info = movie.info()
releases = movie.releases()
# Loop through countries in releases
for c in releases['countries']:
    # if the country abbreviation==US
    if c['iso_3166_1' ] =='US':
        ## save a "certification" key in the info dict with the certification
       info['certification'] = c['certification']

In [ ]:
def get_movie_with_rating(movie_id): 
    '''Adapted from source = https://github.com/celiao/tmdbsimple''' 
    movie = tmdb.Movies(movie_id)
    info = movie.info() 
    
    releases = movie.releases() 
    
    for c in releases['countries']: 
        if c['iso_3166_1'] == 'US': 
            info['certification'] = c['certification']
    return info

In [ ]:
test = get_movie_with_rating("tt0848228") #put your function name here
test

In [ ]:
test_ids = ["tt0848228", "tt0115937","tt0848228","tt0332280"]
results = []
for movie_id in test_ids:
    try:
        movie_info = get_movie_with_rating(movie_id)
        results.append(movie_info)
    except Exception as e: 
        pass
    
pd.DataFrame(results)

In [ ]:
test_ids = ["tt0848228", "tt0115937","tt0848228","tt0332280"]
results = []
errors = []
for movie_id in test_ids:
    
    try:
        movie_info = get_movie_with_rating(movie_id)
        results.append(movie_info)
        
    except Exception as e: 
        errors.append([movie_id, e])
    
pd.DataFrame(results)

In [ ]:
print(f"- Number of errors: {len(errors)}")
errors

In [ ]:
import os, time,json
import tmdbsimple as tmdb 
FOLDER = "Data/"
os.makedirs(FOLDER, exist_ok=True)
os.listdir(FOLDER)

In [ ]:
YEARS_TO_GET = [2000,2001]

In [ ]:
errors = []

In [ ]:
basics = pd.read_csv('Data/title_basics.csv.gz')
basics.head()

In [ ]:
errors = []

In [ ]:
#YEAR = 2000

In [ ]:
for YEAR in tqdm_notebook(YEARS_TO_GET, desc='YEARS', position=0):
    JSON_FILE = f'{FOLDER}tmdb_api_results_{YEAR}.json'
    file_exists = os.path.isfile(JSON_FILE) 
    
    if file_exists == False:
# save an empty dict with just "imdb_id" to the new json file.
        with open(JSON_FILE,'w') as f:
            json.dump([{'imdb_id':0}],f)

    df = basics.loc[ basics['startYear']==YEAR].copy()
# saving movie ids to list
    movie_ids = df['tconst'].copy()

    movie_ids

    previous_df = pd.read_json(JSON_FILE)

    previous_df

    movie_ids_to_get = movie_ids[~movie_ids.isin(previous_df['imdb_id'])]

   # INNER Loop
    for movie_id in tqdm_notebook(movie_ids_to_get,
                                  desc=f'Movies from {YEAR}',
                                  position=1,
                                  leave=True):
        try:
            # Retrieve then data for the movie id
            temp = get_movie_with_rating(movie_id)  
            # Append/extend results to existing file using a pre-made function
            write_json(temp,JSON_FILE)
            # Short 20 ms sleep to prevent overwhelming server
            time.sleep(0.02)
            
        except Exception as e:
            errors.append([movie_id, e])

    final_year_df = pd.read_json(JSON_FILE)
    final_year_df.to_csv(f"{FOLDER}final_tmdb_data_{YEAR}.csv.gz", compression="gzip", index=False)

In [ ]:
print(f"- Total errors: {len(errors)}")

In [ ]:
def write_json(new_data, filename): 
    """Appends a list of records (new_data) to a json file (filename). 
    Adapted from: https://www.geeksforgeeks.org/append-to-json-file-using-python/"""  
    
    with open(filename,'r+') as file:
        # First we load existing data into a dict.
        file_data = json.load(file)
        ## Choose extend or append
        if (type(new_data) == list) & (type(file_data) == list):
            file_data.extend(new_data)
        else:
             file_data.append(new_data)
        # Sets file's current position at offset.
        file.seek(0)
        # convert back to json.
        json.dump(file_data, file)

In [ ]:
avengers = get_movie_with_rating("tt0848228")
avengers

In [ ]:
notebook = get_movie_with_rating("tt0332280")
notebook

In [ ]:
df_2001 = pd.read_csv("Data/final_tmdb_data_2001.csv.gz", low_memory=False)

In [ ]:
df_2001

In [ ]:
df_2000 = pd.read_csv("Data/final_tmdb_data_2000.csv.gz", low_memory=False)

df_2000.head()